# Modulos

In [15]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import matplotlib.pyplot as plt

# Importar datos y Transformar

In [16]:
# Transformaciones para las imágenes (normalización estándar para CIFAR-10)
transform = transforms.Compose(
    [transforms.ToTensor(),  # Convierte imágenes PIL a tensores de PyTorch
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]) # Normaliza los canales RGB

# Dataset de entrenamiento CIFAR-10
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True, num_workers=2)

# Dataset de prueba CIFAR-10
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=32, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

# Arquitectura de CNN

In [17]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        # Capas convolucionales:
        # Conv2d(canales_entrada, canales_salida, tamaño_kernel)
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1) # 3 canales de entrada (RGB), 32 canales de salida, kernel 3x3 y padding
        self.bn1 = nn.BatchNorm2d(32) # Capa de Batch Normalization

        # Gracias al padding conserva el tamaño de salida (ej:5x5 entrada, 5x5 salida)

        self.conv2 = nn.Conv2d(32, 64, 3, padding=1) # 32 canales de entrada (salida de conv1), 64 canales de salida, kernel 3x3
        self.bn2 = nn.BatchNorm2d(64) # Capa de Batch Normalization

        self.conv3 = nn.Conv2d(64, 128, 3, padding=1) # Nueva capa convolucional en el constructor, kernel 3x3
        self.bn3 = nn.BatchNorm2d(128) # Nueva capa de Batch Normalization

        self.conv4 = nn.Conv2d(128, 256, 3, padding=1) # -- Nueva capa convolucional en el constructor, kernel 3x3
        self.bn4 = nn.BatchNorm2d(256) # -- Nueva capa de Batch Normalization

        # Capa de Max Pooling (2x2
        self.pool = nn.MaxPool2d(2, 2)     # Capa de Max Pooling 2x2, reduce dimensionalidad de las caracteristicas extraídas por las capas convolucionales

        # Capas totalmente conectadas (lineales):
        self.fc1 = nn.Linear(256 * 2 * 2 , 512)
        self.fc2 = nn.Linear(512, 128)
        self.fc3 = nn.Linear(128, 10)      # 10 clases de salida (CIFAR-10)

    def forward(self, x):
        # Función de paso hacia adelante: define cómo los datos fluyen por la red
        x = self.pool(F.relu(self.bn1(self.conv1(x)))) # conv1 -> ReLU -> pool
        x = self.pool(F.relu(self.bn2(self.conv2(x)))) # conv2 -> ReLU -> pool
        x = self.pool(F.relu(self.bn3(self.conv3(x)))) # Nueva capa en el Forward
        x = self.pool(F.relu(self.bn4(self.conv4(x)))) # -- Nueva capa en el Forward
        # print(f"Tamaño después de convoluciones y pooling: {x.shape}")

        x = torch.flatten(x, 1) # Aplanar la salida para las capas totalmente conectadas (aplanar desde la dimensión 1 en adelante)

        # print(f"Tamaño después de convoluciones y pooling: {x.shape}")
        x = F.relu(self.fc1(x))           # fc1 -> ReLU
        x = F.relu(self.fc2(x))           # fc2 -> ReLU
        x = self.fc3(x)                   # fc3 (capa de salida, no ReLU aquí típicamente en clasificación)

        return x

# ReLU significa Unidad de Activacion Lineal Rectificada (Rectified Linear Unit).
# Es una funcion de activacion comunmente utiizada en redes neuronales, añade no linealidad a las RN- pertimiendole aprender patrones complejos eficientemente
#     - Si la entrada es mayor que cero, la salida es igual a la entrada.
#     - Si la entrada es menor o igual a cero, la salida es cero.
# BENEFICIOS: - Introduce la NO LINEALIDAD, Eficiencia Computacional -
# - Ayuda a resolver el problema del gradiente desvanecido



net = Net() # Crear una instancia de la red neuronal
dummy_input = torch.randn(1, 3, 32, 32)  # Simulación de una imagen CIFAR-10
output = net(dummy_input) # IMPORTANTE: Usar estas dos líneas siempre

# Simula la entrada de una imagen, si algo estuviera mal en la arquitectura fallaria en el entrenamiento
# Con esas dos lineas el fallo lo encuentra ahora al ejecutar la celda, lo cual es muy valioso y ahorra tiempo

## Opcional: Descarga una imagen de la red **neuronal**

In [18]:
"""
from torchviz import make_dot

dummy_input = torch.randn(1, 3, 32, 32)
output = net(dummy_input)

dot = make_dot(output, params=dict(net.named_parameters()))
dot.format = 'png'
dot.render('cnn_architecture')
"""

"\nfrom torchviz import make_dot\n\ndummy_input = torch.randn(1, 3, 32, 32)\noutput = net(dummy_input)\n\ndot = make_dot(output, params=dict(net.named_parameters()))\ndot.format = 'png'\ndot.render('cnn_architecture')\n"

# Funcion perdida y Optimizador

In [19]:
criterion = nn.CrossEntropyLoss() # Función de pérdida: Cross Entropy Loss (común para clasificación multiclase)
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9, weight_decay=0.001) # Optimizador: SGD (Stochastic Gradient Descent)
# lr: learning rate (tasa de aprendizaje), momentum: para acelerar el descenso

# Entrenamiento

In [20]:
for epoch in range(20):  # Loop sobre el dataset varias veces (épocas)

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0): # Loop sobre batches de datos
        inputs, labels = data      # Obtener las imágenes y las etiquetas del batch
        optimizer.zero_grad()       # Poner a cero los gradientes (importante en cada iteración)

        outputs = net(inputs)       # Paso hacia adelante: obtener las predicciones del modelo
        loss = criterion(outputs, labels) # Calcular la pérdida (diferencia entre predicciones y etiquetas reales)
        loss.backward()             # Paso hacia atrás: calcular los gradientes
        optimizer.step()            # Optimizar: actualizar los pesos del modelo usando los gradientes y el optimizador

        running_loss += loss.item() # Acumular la pérdida para este batch
        if i % 2000 == 1999:    # Imprimir cada 2000 batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

Finished Training


# Precisión Modelo

In [21]:
correct = 0  # Contador predicciones correctas
total = 0    # Contador del Total de Imagenes
# No necesitamos calcular gradientes durante la prueba, así que usamos no_grad() para optimizar
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1) # Obtener la clase predicha (la de mayor probabilidad)
        total += labels.size(0)           # Contar el número total de imágenes
        correct += (predicted == labels).sum().item() # Contar cuántas predicciones fueron correctas

print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')


Accuracy of the network on the 10000 test images: 77 %


# Guardar Modelo/Descargar Modelo

In [22]:
MODEL_SAVE_PATH = '/content/redneuronal.pth' # Extensión .pth para modelos con PyTorch
torch.save(net.state_dict(), MODEL_SAVE_PATH) # Guardar con las funciones de PyTorch
print(f"Modelo guardado en formato PyTorch en: {MODEL_SAVE_PATH}")

Modelo guardado en formato PyTorch en: /content/redneuronal.pth
